In [3]:
!pip install pyproj
!pip install geopandas

     |████████████████████████████████| 6.5MB 5.1MB/s 
     |████████████████████████████████| 972kB 4.8MB/s 
     |████████████████████████████████| 14.8MB 298kB/s 


In [4]:
import warnings
warnings.filterwarnings(action='ignore')
from functools import partial
from shapely import ops
import pyproj
import json, math
import matplotlib.pyplot as plt
from folium import plugins
import re
from matplotlib import font_manager, rc, rcParams
# def set_korea_font():
#     font_name = font_manager.FontProperties(fname="/System/Library/Fonts/Supplemental/AppleGothic.ttf").get_name()
#     rc('font', family=font_name)
#     rcParams.update({'font.size': 11})
#     rcParams['axes.unicode_minus'] = False  
# set_korea_font()
from shapely.geometry import Point as shapely_Point
from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi

from geopy.distance import geodesic
import folium
from folium import plugins
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import os
import datetime

#reproject import 필요

def reproject(geom, from_proj=None, to_proj=None):
    tfm = partial(pyproj.transform, pyproj.Proj(init=from_proj), pyproj.Proj(init=to_proj))
    return ops.transform(tfm, geom)

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
import time
from matplotlib import animation, rc
from IPython.display import HTML
%matplotlib inline


In [5]:
df1 = pd.concat([pd.read_csv('0_1500.csv'), pd.read_csv('1500_2214.csv'), pd.read_csv('2214_4428.csv'), pd.read_csv('4428_6642.csv'), pd.read_csv('6642_8856.csv')])
df1.index = range(len(df1))

#delete nan(선택 feature만)
df1 = df1.loc[~np.isnan(df1.chaos1_nearby_count25)]
df1 = df1.loc[~np.isnan(df1.floating_pop_count50)]

df1

,lon,lat,overspeed_cam_count100,floating_pop_count50,bump_count0,parking_count12.5,parking_count25,car_count1000,child_count1000,elem_kinder_count400,numberSchoolZone_count400,accident_count12.5,accident_count25,chaos1_nearby_count12.5,chaos1_nearby_count25,chaos1_nearby_count50,width_nearby_count12.5,width_nearby_count25,cross_road_nearby_count12.5,cross_road_nearby_count25,num_cram_school_count400,shortest_cross_count0,shortest_traffic_signal_count0,shortest_sidewalk_count0
0,127.061509,37.149281,0,390.898125,143.582568,0,0,470,24.0,1,0,0,1,90.53,69.890000,84.572500,1.0,1.333333,2,3,17,0.000000,4.965855,58.860463
1,127.061615,37.149503,0,507.739167,117.121977,0,0,470,24.0,1,0,0,1,NaN,79.786667,82.809375,1.0,1.750000,1,4,17,19.934105,16.786558,42.782539
2,127.061865,37.149617,0,364.718889,98.525988,0,0,470,24.0,1,0,0,0,84.97,94.676667,79.335000,1.0,2.000000,2,3,26,16.631819,21.477201,33.348896
3,127.060957,37.149433,0,427.077917,154.755132,0,0,470,24.0,1,0,0,0,90.53,96.023750,88.858333,1.0,2.000000,1,3,4,11.360300,9.200259,8.918163
4,127.061237,37.149457,0,476.310556,137.437890,0,0,470,24.0,1,0,0,1,90.53,79.570000,90.390000,1.0,2.000000,1,3,17,24.794651,27.730343,29.463053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8851,127.060176,37.169517,0,65.023333,111.916284,0,0,1576,193.0,4,0,0,0,100.00,98.180000,80.493000,1.0,1.500000,0,0,21,73.965878,77.898523,28.913754
8852,127.060430,37.169633,0,69.116944,137.322563,0,0,1576,193.0,4,0,0,0,100.00,100.000000,72.146667,1.0,1.000000,0,0,21,54.106111,58.622619,31.394445
8853,127.060416,37.169753,0,30.628750,141.076523,0,0,1576,193.0,4,1,0,0,100.00,100.000000,100.000000,1.0,1.000000,0,0,21,60.725272,60.348157,44.209104
8854,127.060214,37.169860,0,63.999583,130.845389,0,0,1576,193.0,4,1,0,0,100.00,100.000000,85.557500,1.0,1.000000,0,0,21,67.094765,65.222149,40.431478


In [6]:
df1.columns

Index(['lon', 'lat', 'overspeed_cam_count100', 'floating_pop_count50',
       'bump_count0', 'parking_count12.5', 'parking_count25', 'car_count1000',
       'child_count1000', 'elem_kinder_count400', 'numberSchoolZone_count400',
       'accident_count12.5', 'accident_count25', 'chaos1_nearby_count12.5',
       'chaos1_nearby_count25', 'chaos1_nearby_count50',
       'width_nearby_count12.5', 'width_nearby_count25',
       'cross_road_nearby_count12.5', 'cross_road_nearby_count25',
       'num_cram_school_count400', 'shortest_cross_count0',
       'shortest_traffic_signal_count0', 'shortest_sidewalk_count0'],
      dtype='object')

In [7]:
df1.loc[2210]

lon                                127.074632
lat                                 37.139593
overspeed_cam_count100               0.000000
floating_pop_count50               108.887917
bump_count0                        211.879738
parking_count12.5                    0.000000
parking_count25                      2.000000
car_count1000                     1909.000000
child_count1000                    422.000000
elem_kinder_count400                13.000000
numberSchoolZone_count400            2.000000
accident_count12.5                   0.000000
accident_count25                     0.000000
chaos1_nearby_count12.5             99.070000
chaos1_nearby_count25               99.070000
chaos1_nearby_count50               87.335000
width_nearby_count12.5               2.000000
width_nearby_count25                 2.000000
cross_road_nearby_count12.5          0.000000
cross_road_nearby_count25            0.000000
num_cram_school_count400            91.000000
shortest_cross_count0             

In [8]:
#feature 선택(사고x, 안전시설물x, 12.5m x)
cols = ["floating_pop_count50",
         "parking_count25",
         "car_count1000", 
         "child_count1000",
         "elem_kinder_count400",
         "numberSchoolZone_count400",
         "chaos1_nearby_count25",
         "width_nearby_count25",
         "cross_road_nearby_count25",
         "num_cram_school_count400",
         "shortest_cross_count0",
         "shortest_traffic_signal_count0"]

df2 = df1.loc[:,cols]

sum(np.isnan(df2.to_numpy()))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
#dbscan
#https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html
#https://scikit-learn.org/stable/modules/clustering.html

df2_s = StandardScaler().fit_transform(df2)


# #hyper param search
# indexs = []
# for min_s in range(7,11):
#   for eps in np.linspace(0.7,1.4,20):
  
#     indexs.append([eps, min_s])

# animate = []
# for min_s in range(7,11):
#   for eps in np.linspace(0.7,1.4,20):
#     db = DBSCAN(eps=eps, min_samples=min_s).fit(df2_s)
#     labels = db.labels_

#     ## Number of clusters in labels, ignoring noise if present.
#     # n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
#     # n_noise_ = list(labels).count(-1)
#     df3 = pd.concat([df2,pd.DataFrame(db.labels_)],axis=1)
#     animate.append(df3.groupby(0).count().floating_pop_count50)

# fig, ax = plt.subplots()

# ax.set_xlim(( -2, 300))
# ax.set_ylim((0, 6000))

# line, = ax.plot([], [], lw=2)

# def init():
#     line.set_data([], [])
#     return (line,)

# def animate_f(i):
#     x = animate[i].index
#     y = animate[i]
#     line.set_data(x, y)
#     ax.set_title('eps, min_samples = {}'.format(indexs[i]))
#     return (line,)

# anim = animation.FuncAnimation(fig, animate_f, init_func=init,
#                                frames=len(indexs), interval=len(indexs), blit=True)

# HTML(anim.to_html5_video())


In [12]:
#1.03, 10 이 적당
db = DBSCAN(eps=1.03, min_samples=10).fit(df2_s)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('eps, min_samples = ',1.03,'/', 10)
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
df3 = pd.concat([df2,pd.DataFrame(db.labels_,index=df2.index)],axis=1)
df3.groupby(0).count().floating_pop_count50

eps, min_samples =  1.03 / 10
Estimated number of clusters: 78
Estimated number of noise points: 2235


0
-1     2235
 0       22
 1     2378
 2       65
 3       23
       ... 
 73      16
 74       6
 75      14
 76       9
 77      11
Name: floating_pop_count50, Length: 79, dtype: int64

In [13]:
#cluster 확인
x = df3.groupby(0).count()
idx = x.loc[x.floating_pop_count50>30].index.to_list()
df4 = df3.loc[df3.loc[:,0].apply(lambda x: x in idx[1:])]
df4.loc[df4.loc[:,0]==25]

,floating_pop_count50,parking_count25,car_count1000,child_count1000,elem_kinder_count400,numberSchoolZone_count400,chaos1_nearby_count25,width_nearby_count25,cross_road_nearby_count25,num_cram_school_count400,shortest_cross_count0,shortest_traffic_signal_count0,0
699,12.742803,0,2819,197.0,1,0,90.730000,2.0,0,0,127.159670,124.128548,25
700,8.766944,0,2819,197.0,1,0,90.730000,2.0,0,0,104.469953,101.394633,25
701,7.575833,0,2819,197.0,1,0,90.730000,2.0,0,0,82.933399,79.845020,25
702,7.395833,0,2819,197.0,1,0,90.730000,2.0,0,0,63.696595,60.729043,25
2673,56.322222,7,2819,197.0,2,0,27.835000,3.0,0,0,92.721208,34.288252,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8843,1.370833,0,2819,197.0,6,1,15.665000,3.0,0,0,45.752908,44.944995,25
8846,40.250354,32,2819,277.0,1,0,45.291667,3.0,0,0,6.109488,10.548649,25
8847,46.451389,62,2819,277.0,1,0,66.415000,3.0,0,0,11.421650,15.391868,25
8848,41.505500,38,2819,277.0,1,0,81.987500,3.0,0,0,36.993217,36.768893,25


In [14]:
#cluster 별로 사고율 확인
sep = []
for i in idx[1:]:
  sep.append(df4.groupby(0).apply(lambda x: x.index)[i].tolist())

print('rows,\t 카메라,\t 유동인구,\t 사고,\t 사고/유동인구,\t 사고/rows')
for i in range(len(sep)):
  print('{},\t {},\t\t {},\t {},\t {},\t\t {}'.format(df1.loc[sep[i]].overspeed_cam_count100.count(),
                                                    df1.loc[sep[i]].overspeed_cam_count100.sum(),
                                                    df1.loc[sep[i]].floating_pop_count50.sum()//1,
                                                    df1.loc[sep[i]].accident_count25.sum(),
                                                    df1.loc[sep[i]].accident_count25.sum()*100000//df1.loc[sep[i]].floating_pop_count50.sum(),
                                                    df1.loc[sep[i]].accident_count25.sum()*1000//df1.loc[sep[i]].floating_pop_count50.count())) 
  

rows,	 카메라,	 유동인구,	 사고,	 사고/유동인구,	 사고/rows
2378,	 73,		 214178.0,	 178,	 83.0,		 74
65,	 2,		 9882.0,	 14,	 141.0,		 215
171,	 33,		 11839.0,	 4,	 33.0,		 23
32,	 0,		 1115.0,	 0,	 0.0,		 0
47,	 0,		 1826.0,	 1,	 54.0,		 21
79,	 25,		 17373.0,	 13,	 74.0,		 164
487,	 11,		 21004.0,	 2,	 9.0,		 4
31,	 0,		 10926.0,	 8,	 73.0,		 258
105,	 0,		 9177.0,	 3,	 32.0,		 28
77,	 10,		 6629.0,	 0,	 0.0,		 0
41,	 0,		 2288.0,	 0,	 0.0,		 0
64,	 0,		 7384.0,	 0,	 0.0,		 0
69,	 4,		 8228.0,	 10,	 121.0,		 144
